# My Local Laplacian
Tim Tyree<br>
6.7.2020<br>

In [6]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from lib.cardiomyocyte import *

## Local Orthogonal Basis that is explicitely computable
- for each lattice site location, $q_{i,j}\in\mathbf{R}^3$, taken from a lattice discretizing a locally smooth, 2D surface of compact support embedded within $\mathbf{R}^3$, 
- we may orient such that $\hat{x}$ is the unit vector in the direction of the nearest neighboring lattice site and $\hat{y}$ is the vector orthogonal to $\hat{x}$ that lies on the smooth manifold such that $\hat{x}\times\hat{y}$ is directed outward.  
    - Notice that local orthogonal basis $(\hat{x},\hat{y})$ is explicitely computable.
- we will consider in what follows a continuous, smooth scalar field, $\psi:\mathbf{R}^3\rightarrow\mathbf{R}$ evaluated at lattice sites chosen from a locally smooth triangular lattice, $\mathcal{L}_\text{triangular}$.

## Approximating the surface - Attempt #1
### Convex Averaging of neighboring weights (--> I'd reckon this is a first order accurate approximation)
Let $q_A\in\mathbf{R}^3$ be the lattice site of the neighbor nearest to $q_{i,j+1}$.
Let $N(q_{i,j})$ be the collection of lattice site locations neighboring $q_{i,j}$ that are not the nearest neighbor of $q_{i,j}$.  Then,<br>
for each $k\in N(q_{i,j})$:<br>
   - let $q(k)\in \mathbf{R}^3$ be the location of the lattice site neighboring $q_{i,j}$.
   - let $\vartheta$ be the angle between $A$ and $B$, where $A$ is the ray beginning at $q_{i,j}$ and extending towards $q_A$ and where $B$ is the ray beginning at $q_{i,j}$ and extending towards $q_k$.
   - let $\Delta^2 x(k) = \Big[(q(k) - q_{i,j})\cdot(q(k) - q_0)\Big]^2$ = the $k^{th}$ square displacement weight in the x direction
   - let $\Delta^2 y(k) = \Big[(q(k) - q_{i,j})\times(q(k) - q_0)\Big]^2$ = the $k^{th}$ square displacement weight in the y direction 
then, take a mean squared average of the neighbors, binning each neighbor into up to 5 of 8 bins representing a square lattice around $q_{i,j}$.<br>
then, take the best approximation a square lattice can have for a 2D laplacian, optimizing for local isotropic symmetry (old news).<br>

- let $\Delta^2 q$ be the mean squared distance of neighboring lattice sites 

Now, the remaining question is, how do we best interpolate the real valued scalar field $\psi(q)$ onto our square lattice?  **See Magnus Expansion--> not helpful**

## Approximating the surface - Attempt #2
### Bezier surface
Let $q_{a,b}$ define the origin of a Bezier surface $\mathbf{p}(u,v)\in[0,1]\times[0,1]$.  Recall then that for control points $mathbf{k}_{i,j}$, we have
$$
\mathbf{p}(u,v) = \sum^n_{i=0} \sum^m_{j=0} B^n_i(u)B^m_j(v)\mathbf{k}_{i,j}
$$

where 
$$
B_i^n(u)= { n \choose k } u^i(1-u)^{n-i}
$$
is a Bernstin polynomial.

__Some properties of Bézier surfaces:__

- A Bézier surface will transform in the same way as its control points under all linear transformations and translations.
- All u = constant and v = constant lines in the (u, v) space, and, in particular, all four edges of the deformed (u, v) unit square are Bézier curves.
- A Bézier surface will lie completely within the convex hull of its control points, and therefore also completely within the bounding box of its control points in any given Cartesian coordinate system.
- The points in the patch corresponding to the corners of the deformed unit square coincide with four of the control points.
- However, a Bézier surface does not generally pass through its other control points.

^this was taken from from https://en.wikipedia.org/wiki/B%C3%A9zier_surface

# Define Helper Functions

In [1]:
def _my_get_laplacian():
    '''compute laplaction from local infinitely smooth fit.
    disp_list'''
    disp_lst, angle_list = _get_local_geometry(self, self.get_neighboring_ids())
    total_angle = np.sum(angle_lst)
    #TODO(see magnus expansion?):  establish the best local 2D coordinates.  suppose locally Lie smoothness.

    #TODO: define an isotropic laplacian with respect to those best local 2D coordinates
    return dV2dx2

def _get_local_geometry(self, neighboring_ids):
    '''assign angular coordinates to neighbors'''
    theta_lst = [0.]
    self.get_neighboring_ids()
    point_1, point_2 = (np.mean(self.triangle, axis=0),np.mean(other.triangle, axis=0))
    point_old = point_2
    disp   = [self.get_displacement_matrix_element(point_1, point_2)]
    if len(neighboring_ids)==0:
        return disp_lst, angle_list
    for nid in neighboring_ids[1:]:
        other  = self.data.loc[nid]
        point_1, point_2 = (np.mean(self.triangle, axis=0),np.mean(other.triangle, axis=0))
        disp   = self.get_displacement_matrix_element(point_1, point_2)
        angle  = _get_angle(q1=point_1, q2=point_old, q3=point_2)
        theta_lst.append(angle)
    return disp_lst, angle_list

def _get_angle(q1, q2, q3):
    '''returns the angle between q2-q1 and q3-q1'''
    return np.angle(q2-q1, q3-q1)